In [63]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [64]:
!pip install -q langchain-openai langchain-core requests

In [65]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [66]:
# tool create

@tool
def multiply(a: int, b: int) -> int:
  '''Given 2 numbers a and b, this tool returns their product'''
  return a*b

In [67]:
print(multiply.invoke({'a':2, 'b':8}))

16


In [68]:
print(f'multiply name: {multiply.name}')
print(f'multiply description: {multiply.description}')

multiply name: multiply
multiply description: Given 2 numbers a and b, this tool returns their product


In [69]:
# tool binding
llm = ChatOpenAI(model='gpt-4o-mini')
llm_with_tools = llm.bind_tools([multiply])

### Tool Calling

It is the process where the LLM decides during a conversation or task, that it needs to use a specific tool and generates a structured output with:

* the name of the tool
* the arguments to call it with

`Note`: The LLM does not actually run the tool, it just suggests the tool and the input arguments. The actual execution is handled by LangChain for you.

In [70]:
llm_with_tools.invoke("Hi, how are you ?")

AIMessage(content="I'm just a computer program, so I don't have feelings, but I'm here and ready to help you! How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 60, 'total_tokens': 89, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CKffIgC8GYcMdCqsSaSl8SDDboflP', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--785f5b37-5a40-4931-a813-f1ace8856ae6-0', usage_metadata={'input_tokens': 60, 'output_tokens': 29, 'total_tokens': 89, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [71]:
query = HumanMessage("Can you multiply 4567 * 34 ?")

In [72]:
messages = [query]

In [73]:
messages

[HumanMessage(content='Can you multiply 4567 * 34 ?', additional_kwargs={}, response_metadata={})]

In [74]:
result = llm_with_tools.invoke(messages)

In [75]:
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0PkgNQeIqKZOHbeAqeX5wepk', 'function': {'arguments': '{"a":4567,"b":34}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 64, 'total_tokens': 82, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CKffJujXjm5anQ7siwTpy9MiOcfNE', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--40ae53b4-d4e4-4f71-8732-95d01f6c6de3-0', tool_calls=[{'name': 'multiply', 'args': {'a': 4567, 'b': 34}, 'id': 'call_0PkgNQeIqKZOHbeAqeX5wepk', 'type': 'tool_call'}], usage_metadata={'input_tokens': 64, 'output_tokens': 18, 'total_tokens': 82, 'input_token_details': {'audio': 

In [76]:
messages.append(result)

In [77]:
messages

[HumanMessage(content='Can you multiply 4567 * 34 ?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0PkgNQeIqKZOHbeAqeX5wepk', 'function': {'arguments': '{"a":4567,"b":34}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 64, 'total_tokens': 82, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CKffJujXjm5anQ7siwTpy9MiOcfNE', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--40ae53b4-d4e4-4f71-8732-95d01f6c6de3-0', tool_calls=[{'name': 'multiply', 'args': {'a': 4567, 'b': 34}, 'id': 'call_0PkgNQeIqKZOHbeAqeX5wepk', 'type': 'tool_call'}], usage_meta

###Tool Cooling

Tool Execution is the step where the actual Python function is run using the input arguments that the LLM suggested during tool calling.


In [78]:
tool_result = multiply.invoke(result.tool_calls[0])

In [79]:
tool_result

ToolMessage(content='155278', name='multiply', tool_call_id='call_0PkgNQeIqKZOHbeAqeX5wepk')

In [80]:
messages.append(tool_result)

In [81]:
messages

[HumanMessage(content='Can you multiply 4567 * 34 ?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0PkgNQeIqKZOHbeAqeX5wepk', 'function': {'arguments': '{"a":4567,"b":34}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 64, 'total_tokens': 82, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CKffJujXjm5anQ7siwTpy9MiOcfNE', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--40ae53b4-d4e4-4f71-8732-95d01f6c6de3-0', tool_calls=[{'name': 'multiply', 'args': {'a': 4567, 'b': 34}, 'id': 'call_0PkgNQeIqKZOHbeAqeX5wepk', 'type': 'tool_call'}], usage_meta

In [82]:
llm_with_tools.invoke(messages).content

'The product of 4567 and 34 is 155,278.'